In [1]:
import cv2
import numpy as np 
import time 
import mediapipe as mp
import math 
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
#create a module

class HandDetector():
    def __init__(self,mode=False,maxHands=2,detectionCon=.5,trackCon=.5):
        self.mode=mode
        self.maxHands=maxHands
        self.detectionCon=detectionCon
        self.trackCon=trackCon
        self.mpHands = mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.detectionCon,self.trackCon)
                                 
        self.mpDraw = mp.solutions.drawing_utils
    def findHands(self,img,draw=True):
        
        
        self.results = self.hands.process(img)
    # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)

            return img
            
    def findPosition(self,img,handNo=0,draw=True):
        lmlist=[]
        if self.results.multi_hand_landmarks:
            myHand=self.results.multi_hand_landmarks[handNo]
            for id , lm in enumerate(myHand.landmark):
                h,w,c=img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id,cx,cy])
                if draw:
                    
                    cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
        return lmlist
                
            
                
        
            

        

In [ ]:
wCam,hCam=648,488
cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
ptime=0
detector=HandDetector(detectionCon=.7)


#this block of code is necessary to get a speaker object 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

#volume.GetMute()
#volume.GetMasterVolumeLevel()
#to get the volume range so that we could use it later we have figuerd out that it ranges from -65.25 to 0.0 
#zero is the max volume value and -65.25 is the minimum
volRange=volume.GetVolumeRange()
#itially set the vulume to the maximum value which is zero 
volume.SetMasterVolumeLevel(0, None)
#the function volume.GetVolumeRange() returns a list cotains the min and the max values for the volume where 
#at 0 index contains the lowest value and at index 1 it contains the max value .
minvol=volRange[0]
maxvol=volRange[1]
#intialize the volume bar with 400 hence it starts at y coordinates equals 400
volBar=400
#intialize the volum as zero not for a specific resone , just regular intialization 
vol=0
#the same as above , we will use it later to put a text contains the percentage value of the volume 
volPer=0
#for all vol... variables we will redefine its range according to the task we need the variable to do  
while True:
    #read every frame captured 
    success,img=cap.read()
    #1.find hand landmarks 
    #this function from the module built before , it returns the hands in the image passed and draw dots and lines 
    
    img_=detector.findHands(img)
    #this function returns a list contains the coordinates for every dot defined above 
    #lm : landmarks 
    lmlist=detector.findPosition(img_,draw=False)
    #the lmlist throughs an error at the begining because the algorithm dosn't detect any hands at the begining 
    #and so the lmlist become empty 
    if len(lmlist)!=0:
        print(lmlist[4],lmlist[8])
        # to pick the fingers we need in this project
        #this line to get the fourth finge coordinates which is the thumb
        #RULE 1 : lmlist at index [4] returns [the point id 0  ,x coordinate 1  , y coordinate 2 ]
        x1,y1=lmlist[4][1],lmlist[4][2]
        #this to get the coordinates of th 8 finger which is the middle finger 
        #the same as RULE 1
        x2,y2=lmlist[8][1],lmlist[8][2]
        #calculate the value at the center of the two points 4 and 8 
        cx,cy=(x1+x2)//2,(y1+y2)//2
        #draw a circle at point 4 
        cv2.circle(img,(x1,y1),10,(255,0,255),cv2.FILLED)
        #draw a circle at point 8 
        cv2.circle(img,(x2,y2),10,(255,0,255),cv2.FILLED)
        #draw a line between them 
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),2)
        #draw a circle in the center 
        cv2.circle(img,(cx,cy),10,(255,0,255),cv2.FILLED)
        #this will calculate the length of each point from th origin 
        lenght=math.hypot(x2-x1,y1-y2)
        #print(length)
        #hand range between 50-300
        #volum range between -65 as min and 0 as max
        #so we have to convert it using numpy , of rescale the values
        #we want ro convert the length which the range of it [50,300] to the range [minvol,maxvol]
        vol=np.interp(lenght,[50,300],[minvol,maxvol])
        volume.SetMasterVolumeLevel(vol, None)
        #the volum bar
        volBar=np.interp(lenght,[50,300],[400,150])
        #the volum percentage
        volPer=np.interp(lenght,[50,300],[0,100])
        
        print(vol)
        #if the length between the two fingers is less than 50 change the circle color to blue 
        if lenght<50:
            
            cv2.circle(img,(cx,cy),15,(0,255,0),cv2.FILLED)
            
    #draw an empty rectangle 
    cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
    #draw a filled rectangel that starts at 50 as the x coordinate and the y coordinate varyies from 50 to 150 
    # depending on the length 
    cv2.rectangle(img,(50,int(volBar)),(85,400),(0,255,0),cv2.FILLED)
    #put the percentage number of the volume under the bar 
    cv2.putText(img,f'{int(volPer)} %',(40,459),
                cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
                
    #calculate the FPS :frame per second 
    ctime=time.time()
    fps=1/(ctime-ptime)
    ptime=ctime
    #put the text of the fps on the image 
    cv2.putText(img,str(int(fps)),(20,50),cv2.FONT_HERSHEY_COMPLEX,3,(255,0,0),3)
    #12. display 
    cv2.imshow('image',img)
    cv2.waitKey(1)

[4, 320, 296] [8, 205, 154]
-30.608364790038912
[4, 315, 296] [8, 207, 158]
-32.56314759408995
[4, 308, 280] [8, 201, 141]
-32.51699911539219
[4, 308, 282] [8, 203, 140]
-32.206335912622436
[4, 293, 262] [8, 182, 125]
-32.279549219939184
[4, 294, 264] [8, 182, 126]
-31.91243541637479
[4, 275, 254] [8, 158, 116]
-31.079192160658664
[4, 276, 253] [8, 160, 117]
-31.645930595498953
[4, 264, 245] [8, 138, 105]
-29.14045773199266
[4, 258, 236] [8, 133, 99]
-29.895933290683104
[4, 259, 232] [8, 132, 98]
-30.11385795687726
[4, 256, 233] [8, 128, 101]
-30.310055553272413
[4, 255, 232] [8, 127, 101]
-30.497082254322592
[4, 251, 231] [8, 118, 104]
-30.30295865368366
[4, 250, 230] [8, 120, 105]
-31.22946224016556
[4, 239, 222] [8, 113, 110]
-34.300013409092955
[4, 239, 222] [8, 111, 112]
-34.25049870883894
[4, 227, 215] [8, 117, 118]
-40.02187845518017
[4, 210, 212] [8, 117, 123]
-44.70290235749522
[4, 210, 209] [8, 117, 125]
-45.591586632794304
[4, 200, 208] [8, 123, 129]
-49.50707430635087
[4, 1

[4, 395, 232] [8, 276, 23]
-15.528550422982903
[4, 398, 234] [8, 275, 24]
-14.780406888897659
[4, 395, 233] [8, 269, 19]
-13.483657060892426
[4, 397, 235] [8, 264, 23]
-12.980610895385745
[4, 398, 235] [8, 263, 23]
-12.701711539095776
[4, 395, 231] [8, 261, 21]
-13.282181088566183
[4, 395, 235] [8, 261, 22]
-12.620761842116345
[4, 395, 232] [8, 262, 22]
-13.42220357472057
[4, 396, 235] [8, 261, 22]
-12.48140935875336
[4, 396, 228] [8, 261, 19]
-13.360808243403582
[4, 396, 232] [8, 261, 20]
-12.701711539095776
[4, 397, 230] [8, 259, 18]
-12.277832904394906
[4, 397, 232] [8, 258, 20]
-12.135084334671745
[4, 398, 229] [8, 258, 18]
-12.20925449807666
[4, 396, 227] [8, 255, 16]
-12.064596762758356
[4, 395, 229] [8, 256, 20]
-12.788480081744403
[4, 394, 225] [8, 254, 18]
-13.076627433104932
[4, 394, 225] [8, 254, 19]
-13.292659214516398
[4, 395, 230] [8, 253, 19]
-11.919228047573895
[4, 396, 230] [8, 253, 21]
-12.204616424443074
[4, 394, 228] [8, 252, 18]
-12.135599118559227
[4, 393, 228] [8